In [1]:
import sqlite3

conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()
    

In [2]:
# query1=Select("Select count(*) from Tips")

In [3]:
# query1.fetchone()

In [6]:
query = Select("Select Reviews.user_id, Reviews.business_id, Reviews.starts, Reviews.review_id, Reviews.review_text,  \
Tips.tip from Reviews join Tips on \
 Reviews.user_id=Tips.user_id and Reviews.business_id=Tips.business_id")

In [7]:
data=[]
i = 0
for d in query:
    i += 1
    data.append(d)
    

In [8]:
len(data)

1000

In [10]:
for d in data[:1]:
    print (d)

('oU2SSOmsp_A8JYI7Z2JJ5w', 'uYHaNptLzDLoV_JZ_MuzUA', 4, 'llmdwOgDReucVoWEry61Lw', "Location is everything and this hotel has it! The reception is inviting and open 24 hours. They are very helpful and have a lot of patience answering all my questions about where to go etc. there is also a lounge open 24 hours with snack-type food. Breakfast is continental-style so if you want heartier fare look elsewhere though you don't have to go far. The bus and train stations are right across the street so it's easy access to the airport or anywhere else you may want to go. Turn uphill to old town or cross the bridge to new town. The room with a view i got was spacious and comfortable though it's a bit of a maze to find it-just follow the signs. The windows are double paned so the room is quiet plus i was on the 5th floor which helps. It's a bit pricey but still one of the best values i found!", 'Awesome location!!')


In [11]:
index1=int(len(data)*0.8)
train_data=data[:index1]
test_data=data[index1:]
del data

In [12]:
# print len(train_data)
# print len(test_data)

In [13]:
user_rating_dict={}
item_rating_dict={}

In [14]:
for d in train_data:
    user,item,rating,review_id,text=d[0],d[1],d[2],d[3],d[4]
    if user not in user_rating_dict:
        user_rating_dict[user]=[[] for _ in range(5)]
    if item not in item_rating_dict:
        item_rating_dict[item]=[[] for _ in range(5)]
        
    user_rating_dict[user][int(rating)-1].append(text)
    item_rating_dict[item][int(rating)-1].append(text)    

In [ ]:
user_item_review={}

In [23]:
from sumy.parsers.plaintext import PlaintextParser 
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer 

In [33]:
re = []
#  i = 0
i = 0
tips = []
for d in test_data:
    user,item,rating,tip=d[0],d[1],d[2],d[5]
#     print(user,item,rating)
    user_reveiws=user_rating_dict.get(user,[[] for _ in range(5)])[int(rating)-1]
    item_reviews=item_rating_dict.get(item,[[] for _ in range(5)])[int(rating)-1]
    reviews=user_reveiws+item_reviews
    if len(reviews) == 0:
        continue
    parser = PlaintextParser.from_string(reviews,Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, 1)
    re += [summary]
    tips += [tip]

r-zUIQPaHzvIyL93wQaoiQ QGRpyQOwtl1Dkja-_VM7PA 3
qyeALt2bQl6lMjWLwjNpVw FLdfpoqGGgm_d_xHDe0_Lw 1
U0WU9d6gpMNkO_tPXK58Sg FLdfpoqGGgm_d_xHDe0_Lw 5
0uALWZDD4Hu3_8fykVbL7w FLdfpoqGGgm_d_xHDe0_Lw 1
ejcOUtOlD0WFqPa65HE80Q IPg5ZmRfG6nibu7GQe3LYg 4
6MM9Yqn7UBM8tmpSHQHAAg IPg5ZmRfG6nibu7GQe3LYg 3
uZZRzA-Lem4edf_K34yB7g lfuXIpZ_WxIbxJxUjRTBgg 5
RTElp42I8CzRWNhe5gp-iw lfuXIpZ_WxIbxJxUjRTBgg 5
JcirZUQLHN7cvUcmUtUeCg lfuXIpZ_WxIbxJxUjRTBgg 4
SPZ9OrRGk8dJ7ep8JmoS5g lfuXIpZ_WxIbxJxUjRTBgg 3
FmNuihINhD2lNQLnfFhpnA QLQrOULdkOYB2rsFqgKz3Q 4
XYZw8rWQ0wjMWQChRCU8Hg 86wtLylJ9fbVuXRy5WU3Zw 4
YttDgOC9AlM4HcAlDsbB2A kQbo5EmBXphXfLFddMqsug 5
rcj-kHO5vfiMgRKee7r_xA kQbo5EmBXphXfLFddMqsug 5
C0aL4tgH7g8DpCJS4FmogQ kQbo5EmBXphXfLFddMqsug 5
KdxrZutJCb9ToArv6jCE1A pGHyYl4GrwSg-BAu3laYsw 5
BQwMbDiPiPSPMOru0E8nrQ pGHyYl4GrwSg-BAu3laYsw 2
URYCEpS1oRwBopLX7EjD9A pGHyYl4GrwSg-BAu3laYsw 5
URYCEpS1oRwBopLX7EjD9A pGHyYl4GrwSg-BAu3laYsw 5
CDHORt_NH2R6ySLHZ4npmg jQsNFOzDpxPmOurSWCg1vQ 2


In [36]:
re,"\n",tip

([(<Sentence: I handed it back and said "It\'s dirty, I\'ll take a different one.">,)],
 '\n',
 'Fast, affordable, delicious with many options')

In [ ]:
# len(user_item_review.keys())

In [ ]:
# user_item_tips={}

In [ ]:
# k=0
# for (u,i) in user_item_review:
#     parser = PlaintextParser.from_string(user_item_review[(u,i)],Tokenizer("english"))
#     summarizer = LexRankSummarizer()
#     summary = summarizer(parser.document, 1)
#     user_item_tips[(u,i)]=summary

In [ ]:
# len(user_item_tips.items())

In [37]:
# item_rating_dict['QGRpyQOwtl1Dkja-_VM7PA'][2]